In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from pathlib import Path
import os
from datetime import datetime
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
print('sqlachemy version: ',sqlalchemy.__version__)
import os
from dotenv import load_dotenv
#load_dotenv()

sqlachemy version:  1.4.39


In [79]:
if __name__ == '__main__':

    # Loading env variables
    load_dotenv(dotenv_path=Path('.\.env'))

    user = os.getenv('user')
    password = os.getenv('password')
    host = os.getenv('host')
    #port = os.getenv('port')
    database = os.getenv('database')
    print("database: ",database)

database:  ecommerce


In [80]:
# Creating Postgre SQL connection, passing user and password as arguments when running script
con_string=f'mysql+pymysql://{user}:{password}@{host}/{database}'
engine = create_engine(con_string)

In [2]:
fact_sales_cleaned = pd.read_csv('fact_sales_cleaned.csv')

In [3]:
fact_sales_cleaned.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850,Dog and Puppy Pads,85123A,536365,1,15.30,15.30
1,12/1/2020 8:34,13047,Pet Odor Eliminator,84879,536367,6,60.84,10.14
2,12/1/2020 8:34,13047,Rechargeable Pet Nail Grinder,22960,536368,1,25.50,25.50
3,12/1/2020 9:00,13748,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371,14,214.20,15.30
4,12/1/2020 9:02,17850,Dog and Puppy Pads,85123A,536373,1,15.30,15.30


In [4]:
fact_sales_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24404 entries, 0 to 24403
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  24404 non-null  object 
 1   customer_id       24404 non-null  int64  
 2   description       24404 non-null  object 
 3   stock_code        24404 non-null  object 
 4   invoice_no        24404 non-null  int64  
 5   quantity          24404 non-null  int64  
 6   sales             24404 non-null  float64
 7   unit_price        24404 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 1.5+ MB


In [5]:
customers = pd.read_csv('customers.csv')

In [6]:
customers.head()

,customer_id,order_city,order_postal,order_state,latitude,longitude
0,18287,Rock Springs,82901-7321,WY,41.5869,-109.2048
1,18283,JACKSON,83001-9460,WY,43.4800,-110.7618
2,18282,Greybull,82426,WY,44.4892,-108.0562
3,18281,EVANSTON,82930-4706,WY,41.2680,-110.9633
4,18280,CHEYENNE,82001-7901,WY,41.1400,-104.8202


In [7]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customer_id   4372 non-null   int64  
 1   order_city    4372 non-null   object 
 2   order_postal  4372 non-null   object 
 3   order_state   4372 non-null   object 
 4   latitude      4274 non-null   float64
 5   longitude     4274 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 205.1+ KB


In [8]:
customers['order_state'].nunique()

212

In [9]:
state_region_mapping = pd.read_csv('state_region_mapping.csv')

In [10]:
state_region_mapping.head()

,ï»¿order_state,state,region
0,AE,AE,Other
1,ak,AK,Other
2,AK,AK,Other
3,AL,AL,East
4,ALABAMA,AL,East


In [11]:
state_region_mapping.rename(columns = {'ï»¿order_state':'order_state'}, inplace = True)

In [12]:
state_region_mapping.head()

,order_state,state,region
0,AE,AE,Other
1,ak,AK,Other
2,AK,AK,Other
3,AL,AL,East
4,ALABAMA,AL,East


In [13]:
state_region_mapping.order_state.nunique()

192

In [14]:
state_region_mapping['region'].value_counts()

East       74
Central    71
West       34
Other      13
Name: region, dtype: int64

In [15]:
customers = pd.merge(customers, state_region_mapping, how="left", on=["order_state"])

In [16]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4372 entries, 0 to 4371
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customer_id   4372 non-null   int64  
 1   order_city    4372 non-null   object 
 2   order_postal  4372 non-null   object 
 3   order_state   4372 non-null   object 
 4   latitude      4274 non-null   float64
 5   longitude     4274 non-null   float64
 6   state         4350 non-null   object 
 7   region        4350 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 307.4+ KB


In [17]:
customers[customers.state.isnull()]['order_state'].unique()

array(['VERMONT', 'ri', 'Ne', 'N.C.', 'mo', 'Mississippi', 'Md', 'Ks',
       'Kansas', 'IL.', 'Ia', 'georgia', 'ga', 'florida', 'Ct', 'Az.',
       'Az', 'ALASKA', 'Alabama', 'Al'], dtype=object)

In [18]:
customers[customers['state'].isna()]

,customer_id,order_city,order_postal,order_state,latitude,longitude,state,region
262,17928,HYDE PARK,05655-9361,VERMONT,44.5939,-72.6165,NaN,NaN
828,17148,cranston,2920,ri,41.7810,-71.4371,NaN,NaN
1821,15789,Holdrege,68949,Ne,40.4403,-99.3698,NaN,NaN
1952,15612,Linwood,27299,N.C.,35.7500,-80.3500,NaN,NaN
1990,15565,wentzville,63385,mo,38.8114,-90.8529,NaN,NaN
1992,15562,st. louis,63117,mo,38.6300,-90.1900,NaN,NaN
2116,15396,Carriere,39426,Mississippi,30.6169,-89.6526,NaN,NaN
2306,15143,Arnold,21012,Md,39.0321,-76.5025,NaN,NaN
2566,14785,Clay Center,67432,Ks,39.3769,-97.1247,NaN,NaN
2573,14776,Leavenworth,66048-4189,Kansas,39.3111,-94.9228,NaN,NaN


In [19]:
state_dict={'VERMONT':'VT', 'ri':'RI', 'Ne':'NE', 'N.C.':'NC', 'mo':'MO', 'Mississippi':'MI', 'Md':'MD', 'Ks':'KS',
       'Kansas':'KS', 'IL.':'IL', 'Ia':'IA', 'georgia':'GA', 'ga':'GA', 'florida':'FL', 'Ct':'CT', 'Az.':'AZ',
       'Az':'AZ', 'ALASKA':'AK', 'Alabama':'AL', 'Al':'AL'}

In [ ]:
#get index of these null values

In [20]:
print(list(customers[customers['state'].isna()].index.values))

[262, 828, 1821, 1952, 1990, 1992, 2116, 2306, 2566, 2573, 2680, 2896, 2922, 3019, 3035, 3056, 3414, 4111, 4195, 4224, 4229, 4244]


In [59]:
df_null_st = customers[customers['state'].isna()]

In [61]:
for i in list(customers[customers['state'].isna()].index.values):
    #print(i)
    l_state = customers._get_value(i, 'order_state')
    value_st = state_dict[l_state]
    #print(value_st)
    df_null_st.at[i,'state'] = f'{value_st}'
    #customers[customers['state'].isna()].at[i,'state'] = value_st
   

In [62]:
df_null_st

,customer_id,order_city,order_postal,order_state,latitude,longitude,state,region
262,17928,HYDE PARK,05655-9361,VERMONT,44.5939,-72.6165,VT,NaN
828,17148,cranston,2920,ri,41.7810,-71.4371,RI,NaN
1821,15789,Holdrege,68949,Ne,40.4403,-99.3698,NE,NaN
1952,15612,Linwood,27299,N.C.,35.7500,-80.3500,NC,NaN
1990,15565,wentzville,63385,mo,38.8114,-90.8529,MO,NaN
1992,15562,st. louis,63117,mo,38.6300,-90.1900,MO,NaN
2116,15396,Carriere,39426,Mississippi,30.6169,-89.6526,MI,NaN
2306,15143,Arnold,21012,Md,39.0321,-76.5025,MD,NaN
2566,14785,Clay Center,67432,Ks,39.3769,-97.1247,KS,NaN
2573,14776,Leavenworth,66048-4189,Kansas,39.3111,-94.9228,KS,NaN


In [65]:
NaNfree_cusotmers = customers[~customers['state'].isna()]

In [67]:
NaNfree_cusotmers.shape

(4350, 8)

In [68]:
final_customers_clean_df = NaNfree_cusotmers.append(df_null_st)

C:\Users\anita\AppData\Local\Temp\ipykernel_33116\1257054781.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_customers_clean_df = NaNfree_cusotmers.append(df_null_st)


In [69]:
final_customers_clean_df.shape

(4372, 8)

In [70]:
final_customers_clean_df.head()

,customer_id,order_city,order_postal,order_state,latitude,longitude,state,region
0,18287,Rock Springs,82901-7321,WY,41.5869,-109.2048,WY,West
1,18283,JACKSON,83001-9460,WY,43.4800,-110.7618,WY,West
2,18282,Greybull,82426,WY,44.4892,-108.0562,WY,West
3,18281,EVANSTON,82930-4706,WY,41.2680,-110.9633,WY,West
4,18280,CHEYENNE,82001-7901,WY,41.1400,-104.8202,WY,West


In [71]:
final_customers_clean_df[final_customers_clean_df['state'].isna()]

,customer_id,order_city,order_postal,order_state,latitude,longitude,state,region


In [72]:
final_customers_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4372 entries, 0 to 4244
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customer_id   4372 non-null   int64  
 1   order_city    4372 non-null   object 
 2   order_postal  4372 non-null   object 
 3   order_state   4372 non-null   object 
 4   latitude      4274 non-null   float64
 5   longitude     4274 non-null   float64
 6   state         4372 non-null   object 
 7   region        4350 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 307.4+ KB


In [73]:
#saving this file
csv_name = "final_customers_clean.csv"
final_customers_clean_df.to_csv(csv_name, index=False)

In [81]:
#tbl_list=  ['tbl_customers','tbl_products','tbl_fact_sales','tbl_state_region_mapping']
def get_query_string(table):
    query_string="select * from {}".format(table)
    query="""
            {}
            """.format(query_string)
    return query

In [151]:
df_fact_sales = pd.read_sql(get_query_string('tbl_fact_sales'),engine)

In [152]:
df_fact_sales.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30


In [106]:
df_fact_sales_1= df_fact_sales[df_fact_sales['invoice_no'].notna()]

In [107]:
df_fact_sales_1.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30


In [112]:
df_fact_sales_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24404 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  24404 non-null  object 
 1   customer_id       19996 non-null  float64
 2   description       24404 non-null  object 
 3   stock_code        24404 non-null  object 
 4   invoice_no        24404 non-null  object 
 5   quantity          24404 non-null  int64  
 6   sales             24404 non-null  float64
 7   unit_price        24404 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.7+ MB


In [113]:
df_fact_sales_1['invoice_no'] = df_fact_sales_1['invoice_no'].apply(lambda x: x if pd.isnull(x) else str(int(x)))

C:\Users\anita\AppData\Local\Temp\ipykernel_33116\3171865490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fact_sales_1['invoice_no'] = df_fact_sales_1['invoice_no'].apply(lambda x: x if pd.isnull(x) else str(int(x)))


In [114]:
df_fact_sales_1.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373,1,15.30,15.30


In [117]:
df_fact_sales_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24404 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  24404 non-null  object 
 1   customer_id       19996 non-null  float64
 2   description       24404 non-null  object 
 3   stock_code        24404 non-null  object 
 4   invoice_no        24404 non-null  object 
 5   quantity          24404 non-null  int64  
 6   sales             24404 non-null  float64
 7   unit_price        24404 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.7+ MB


In [124]:
nanfree_sales = df_fact_sales_1[~df_fact_sales_1['customer_id'].isna()]

In [125]:
nanfree_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19996 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  19996 non-null  object 
 1   customer_id       19996 non-null  float64
 2   description       19996 non-null  object 
 3   stock_code        19996 non-null  object 
 4   invoice_no        19996 non-null  object 
 5   quantity          19996 non-null  int64  
 6   sales             19996 non-null  float64
 7   unit_price        19996 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.4+ MB


In [126]:
# Using apply(str) to lambda function
nanfree_sales["customer_id"] = nanfree_sales["customer_id"].astype(int).astype(str)

C:\Users\anita\AppData\Local\Temp\ipykernel_33116\2089020928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nanfree_sales["customer_id"] = nanfree_sales["customer_id"].astype(int).astype(str)


In [127]:
nanfree_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19996 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  19996 non-null  object 
 1   customer_id       19996 non-null  object 
 2   description       19996 non-null  object 
 3   stock_code        19996 non-null  object 
 4   invoice_no        19996 non-null  object 
 5   quantity          19996 non-null  int64  
 6   sales             19996 non-null  float64
 7   unit_price        19996 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.4+ MB


In [129]:
nanfree_sales.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850,Dog and Puppy Pads,85123A,536365,1,15.30,15.30
1,12/1/2020 8:34,13047,Pet Odor Eliminator,84879,536367,6,60.84,10.14
2,12/1/2020 8:34,13047,Rechargeable Pet Nail Grinder,22960,536368,1,25.50,25.50
3,12/1/2020 9:00,13748,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371,14,214.20,15.30
4,12/1/2020 9:02,17850,Dog and Puppy Pads,85123A,536373,1,15.30,15.30


### Tackle customerId Nan data

In [130]:
nancusotmers = df_fact_sales_1[df_fact_sales_1['customer_id'].isna()]

In [131]:
nancusotmers.shape

(4408, 8)

In [132]:
nancusotmers.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
74,12/1/2020 14:32,NaN,ChomChom Pet Hair Remover - Reusable Cat and D...,22114,536544,1,50.82,50.82
75,12/1/2020 14:32,NaN,Dog and Puppy Pads,85123A,536544,1,35.46,35.46
76,12/1/2020 14:32,NaN,Earth Rated Dog Poop Bags,85099B,536544,1,25.26,25.26
77,12/1/2020 14:32,NaN,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536544,6,212.76,35.46
78,12/1/2020 14:32,NaN,Pet Grooming Brush,20725,536544,1,25.26,25.26


In [133]:
nancusotmers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408 entries, 74 to 25049
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  4408 non-null   object 
 1   customer_id       0 non-null      float64
 2   description       4408 non-null   object 
 3   stock_code        4408 non-null   object 
 4   invoice_no        4408 non-null   object 
 5   quantity          4408 non-null   int64  
 6   sales             4408 non-null   float64
 7   unit_price        4408 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 309.9+ KB


In [137]:
nancusotmers.customer_id.fillna('GUEST_'+nancusotmers.invoice_no, inplace=True)

C:\Users\anita\AppData\Local\Temp\ipykernel_33116\1393405156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nancusotmers.customer_id.fillna('GUEST_'+nancusotmers.invoice_no, inplace=True)


In [138]:
nancusotmers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408 entries, 74 to 25049
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  4408 non-null   object 
 1   customer_id       4408 non-null   object 
 2   description       4408 non-null   object 
 3   stock_code        4408 non-null   object 
 4   invoice_no        4408 non-null   object 
 5   quantity          4408 non-null   int64  
 6   sales             4408 non-null   float64
 7   unit_price        4408 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 309.9+ KB


In [139]:
nancusotmers.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
74,12/1/2020 14:32,GUEST_536544,ChomChom Pet Hair Remover - Reusable Cat and D...,22114,536544,1,50.82,50.82
75,12/1/2020 14:32,GUEST_536544,Dog and Puppy Pads,85123A,536544,1,35.46,35.46
76,12/1/2020 14:32,GUEST_536544,Earth Rated Dog Poop Bags,85099B,536544,1,25.26,25.26
77,12/1/2020 14:32,GUEST_536544,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536544,6,212.76,35.46
78,12/1/2020 14:32,GUEST_536544,Pet Grooming Brush,20725,536544,1,25.26,25.26


In [140]:
final_sales_clean_df = nanfree_sales.append(nancusotmers)

C:\Users\anita\AppData\Local\Temp\ipykernel_33116\2966016861.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_sales_clean_df = nanfree_sales.append(nancusotmers)


In [141]:
final_sales_clean_df.shape

(24404, 8)

In [142]:
final_sales_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24404 entries, 0 to 25049
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  24404 non-null  object 
 1   customer_id       24404 non-null  object 
 2   description       24404 non-null  object 
 3   stock_code        24404 non-null  object 
 4   invoice_no        24404 non-null  object 
 5   quantity          24404 non-null  int64  
 6   sales             24404 non-null  float64
 7   unit_price        24404 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.7+ MB


In [144]:
final_sales_clean_df.tail()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
25045,12/9/2021 10:26,GUEST_581498,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,581498,9,312.66,34.74
25046,12/9/2021 10:26,GUEST_581498,Milk-Bone MaroSnacks Dog Treats with Real Bone,22386,581498,1,24.78,24.78
25047,12/9/2021 10:26,GUEST_581498,ProBiotic Supplements for Dogs,79321,581498,2,149.52,74.76
25048,12/9/2021 10:26,GUEST_581498,Purina ONE SmartBlend Natural Adult Chicken 10lb,23298,581498,1,64.74,64.74
25049,12/9/2021 10:26,GUEST_581498,Rechargeable Pet Nail Grinder,22960,581498,1,49.74,49.74


In [146]:
final_sales_clean_df.reset_index(drop=True, inplace=True)

In [154]:
final_sales_clean_df.shape

(24404, 8)

In [149]:
#saving this file
csv_name = "final_sales_clean_df.csv"
final_sales_clean_df.to_csv(csv_name, index=False)

## Tackling no invoice_no details

In [155]:
no_invoice_df = df_fact_sales[df_fact_sales['invoice_no'].isna()]

In [156]:
no_invoice_df.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
54,12/1/2020 12:38,17897.0,Rechargeable Pet Nail Grinder,22960,NaN,-1,-25.5,25.5
195,12/2/2020 17:27,15384.0,Sheba Perfect Portions Pat Wet Cat Food,22197,NaN,0,0.0,5.1
448,12/6/2020 12:55,14487.0,Taste of the Wild High Prairie Grain-Free Dry ...,22423,NaN,0,0.0,76.5
454,12/6/2020 13:17,16550.0,Sheba Perfect Portions Pat Wet Cat Food,22197,NaN,0,0.0,5.1
491,12/6/2020 15:09,16861.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,NaN,-1,-17.7,17.7


In [159]:
no_invoice_df[no_invoice_df['quantity']<0]

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
54,12/1/2020 12:38,17897.0,Rechargeable Pet Nail Grinder,22960,NaN,-1,-25.50,25.50
491,12/6/2020 15:09,16861.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,NaN,-1,-17.70,17.70
683,12/8/2020 13:33,14299.0,Taste of the Wild High Prairie Grain-Free Dry ...,22423,NaN,-1,-65.70,65.70
930,12/10/2020 15:01,13798.0,Earth Rated Dog Poop Bags,85099B,NaN,-2,-19.80,9.90
1077,12/14/2020 9:54,13802.0,Taste of the Wild High Prairie Grain-Free Dry ...,22423,NaN,-1,-65.70,65.70
...,...,...,...,...,...,...,...,...
23659,11/28/2021 15:43,12657.0,Taste of the Wild High Prairie Grain-Free Dry ...,22423,NaN,-1,-76.50,76.50
24041,12/1/2021 8:44,13078.0,Sheba Perfect Portions Pat Wet Cat Food,22197,NaN,-2,-8.64,4.32
24608,12/6/2021 11:40,12753.0,Templation Soft Cat Treats,23084,NaN,-1,-10.74,10.74
24879,12/8/2021 10:06,16019.0,Taste of the Wild High Prairie Grain-Free Dry ...,22423,NaN,-1,-65.70,65.70


In [162]:
no_invoice_df[no_invoice_df['customer_id'].isna()]

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
1582,12/21/2020 16:31,NaN,UltraSound Dog Barking Deterrent,22720,NaN,0,0.00,29.70
5499,3/25/2021 10:43,NaN,Pet Odor Eliminator,84879,NaN,-1,-10.14,10.14
11101,6/21/2021 17:59,NaN,"Memory Foam Pet Beds for Small, Medium, and La...",47566,NaN,-1,-29.70,29.70
11102,6/21/2021 17:59,NaN,Pet Grooming Brush,20725,NaN,-2,-19.80,9.90
11103,6/21/2021 17:59,NaN,Pet Odor Eliminator,84879,NaN,-4,-40.56,10.14
11469,6/28/2021 15:20,NaN,Rechargeable Pet Nail Grinder,22960,NaN,0,0.00,49.74
18987,10/14/2021 12:02,NaN,Pet Grooming Brush,20725,NaN,-1,-9.90,9.90
23002,11/22/2021 17:31,NaN,ChomChom Pet Hair Remover - Reusable Cat and D...,22114,NaN,-8,-180.00,22.50


In [160]:
no_invoice_df[no_invoice_df['quantity']==0]

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
195,12/2/2020 17:27,15384.0,Sheba Perfect Portions Pat Wet Cat Food,22197,NaN,0,0.0,5.10
448,12/6/2020 12:55,14487.0,Taste of the Wild High Prairie Grain-Free Dry ...,22423,NaN,0,0.0,76.50
454,12/6/2020 13:17,16550.0,Sheba Perfect Portions Pat Wet Cat Food,22197,NaN,0,0.0,5.10
492,12/6/2020 15:12,13481.0,Rechargeable Pet Nail Grinder,22960,NaN,0,0.0,25.50
544,12/7/2020 12:45,17511.0,Dog and Puppy Pads,85123A,NaN,0,0.0,15.30
...,...,...,...,...,...,...,...,...
24909,12/8/2021 11:42,16933.0,NexGard Chew for Dogs,82484,NaN,0,0.0,47.70
24944,12/8/2021 14:08,12476.0,NexGard Chew for Dogs,82484,NaN,0,0.0,47.70
25002,12/8/2021 18:48,13078.0,Sheba Perfect Portions Pat Wet Cat Food,22197,NaN,0,0.0,4.32
25004,12/8/2021 19:20,13883.0,UltraSound Dog Barking Deterrent,22720,NaN,0,0.0,29.70


In [164]:
#saving this file
csv_name = "no_invoice_df.csv"
no_invoice_df.to_csv(csv_name, index=False)